In [ ]:
# Cargar autenticación de Google para poder cargar los archivos
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Importación de librerías y dependencias

from sklearn import preprocessing
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [ ]:
steps = ['Reconocimiento','Armamento' 'Delivery', 'Explotación', 'Instalación', 'Comando y Control', 'Acciones']

In [ ]:
id = '1NTzqUa52i-pZ5KXN5XLz9Qn32ejikxJE'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('alerts_dataset.csv')  
df = pd.read_csv('alerts_dataset.csv')

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.dropna()

In [ ]:
df['src_ip'] = df['src_ip'].astype('str')
df['infected_host'] = df['infected_host'].astype('str') 
df['dest_ip'] = df['dest_ip'].astype('str') 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#df['alert_type'] = le.fit_transform(df.alert_type.values)
df['src_ip'] = le.fit_transform(df.src_ip.values)
df['dest_ip'] = le.fit_transform(df.dest_ip.values)
df['infected_host'] = le.fit_transform(df.infected_host.values)
#df['step'] = le.fit_transform(df.step.values)

In [ ]:
df = df.drop(['alert_id','alert_type'], axis=1)
#df = df.drop(['src_ip','infected_host', 'timestamp'], axis=1)

In [ ]:
X = df.drop('step', axis=1)
#y = pd.get_dummies(df.step)
y = df['step']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42, stratify=y)

In [ ]:
#Random Forest
rf = RandomForestClassifier(criterion='gini', n_estimators = 70, random_state = 42, min_samples_split=10, min_samples_leaf=1, max_features='auto', oob_score=True, n_jobs=-1)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
acc_random_forest_test = round(rf.score(X_test, y_test) * 100, 8)
acc_random_forest_test

In [ ]:
acc_random_forest_train = round(rf.score(X_train, y_train) * 100, 4)
print('Accuracy: ', acc_random_forest_train , '%')

In [ ]:
#CrossValidation

skf = StratifiedKFold(n_splits=5, random_state=42)
rf = RandomForestClassifier(criterion='gini', n_estimators = 70, random_state = 42, min_samples_split=10, min_samples_leaf=1, max_features='auto', oob_score=True, n_jobs=-1)

def train_model(fold_no):
  rf.fit(X_train,y_train)
  predictions = rf.predict(X_test)
  print('Fold',str(fold_no),'Accuracy:',round(rf.score(X_test, y_test) * 100, 4))

fold_no = 1
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  train_model(fold_no)
  fold_no += 1

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
feature_list = list(df.columns)

importances = list(rf.feature_importances_)#Lista de tuplas con variable e importancia
feature_importances = [(feature, round(importance, 8)) for feature, importance in zip(feature_list, importances)]# Lista de tuplas con variable e importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Ordenar la lista de importancia de mayor a menor
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];# Imprimir la tupla

In [ ]:
print(classification_report(y_test,y_pred,digits=8))

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# KNN
kn = KNeighborsClassifier()

kn.fit(X_train, y_train)
y_pred = kn.predict(X_test)

In [ ]:
acc_random_forest_test = round(kn.score(X_test, y_test) * 100, 8)
acc_random_forest_test

In [ ]:
acc_random_forest_train = round(kn.score(X_train, y_train) * 100, 8)
acc_random_forest_train

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42)
cv_kn = KNeighborsClassifier()

def train_model(fold_no):
  cv_kn.fit(X_train,y_train)
  predictions = cv_kn.predict(X_test)
  print('Fold',str(fold_no),'Accuracy:',round(cv_kn.score(X_test, y_test) * 100, 4))

fold_no = 1
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  train_model(fold_no)
  fold_no += 1

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Gaussian NB
clf = GaussianNB()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
acc_random_forest_test = round(clf.score(X_test, y_test) * 100, 8)
acc_random_forest_test

In [ ]:
acc_random_forest_train = round(clf.score(X_train, y_train) * 100, 8)
acc_random_forest_train

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42)
#y = df.loc[:,'Returned_Units']

clf = GaussianNB()

def train_model(fold_no):
  clf.fit(X_train,y_train)
  predictions = clf.predict(X_test)
  print('Fold',str(fold_no),'Accuracy:',round(clf.score(X_test, y_test) * 100, 4))

fold_no = 1
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  train_model(fold_no)
  fold_no += 1

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Decision Tree

dt = DecisionTreeClassifier(random_state=42, min_samples_split=23, min_samples_leaf=5)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
y_pred_train = dt.predict(X_train)
pred_prob = dt.predict_proba(X_test)

acc_random_forest_train = round(dt.score(X_train, y_train) * 100, 4)
print('Train Accuracy: ', acc_random_forest_train , '%')

acc_random_forest_test = round(dt.score(X_test, y_test) * 100, 4)
print('Test Accuracy: ', acc_random_forest_test , '%')

In [ ]:
feature_list = list(df.columns)

importances = list(dt.feature_importances_)#Lista de tuplas con variable e importancia
feature_importances = [(feature, round(importance, 8)) for feature, importance in zip(feature_list, importances)]# Lista de tuplas con variable e importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Ordenar la lista de importancia de mayor a menor
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];# Imprimir la tupla

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

def train_model(fold_no):
  dt.fit(X_train,y_train)
  predictions = dt.predict(X_test)
  print('Fold',str(fold_no),'Accuracy:',round(dt.score(X_test, y_test) * 100, 4))

fold_no = 1
for train_index, test_index in skf.split(X, y):
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  train_model(fold_no)
  fold_no += 1

In [ ]:
print(confusion_matrix(y_test,y_pred))

In [ ]:
print(classification_report(y_test,y_pred,digits=8))

In [ ]:
#Confusion matrix all classes
matrix = confusion_matrix(y_test, y_pred)
cm = matrix.diagonal()/matrix.sum(axis=1)
cm

In [ ]:
#Accuracies of each class
pos = 0
target_names = ['S1', 'S2', 'S3', 'S4', 'S5']
print('Class : Accuracy')
for i in cm:
  print(target_names[pos],': ' ,i)
  pos = pos + 1